In [1]:
from sklearn import metrics
import processing
import time
import os
from tqdm import tqdm
from torch.optim import Adam
from torch import nn
import numpy as np
import sys
sys.path.append("D:/Experiment")
import torch
from MyKu import processing
from MyKu import MyXLM
import pandas as pd

In [4]:

# 训练准备阶段，设置超参数和全局变量
file_name = 'readme.md'
batch_size = 32
num_epoch = 50  # 训练轮次
check_step = 1  # 用以训练中途对模型进行检验：每check_step个epoch进行一次测试和保存模型

learning_rate = 1e-5  # 优化器的学习率

# 获取训练、测试数据、分类类别总数
en_train_data, en_test_data, de_train_data, de_test_data, hi_train_data, hi_test_data = processing.load_hasoc2020()
categories = 2

en_train_iter, en_test_iter = MyXLM.load_xlm_data(en_train_data, en_test_data, batch_size)
de_train_iter, de_test_iter = MyXLM.load_xlm_data(de_train_data, de_test_data, batch_size)
hi_train_iter, hi_test_iter = MyXLM.load_xlm_data(hi_train_data, hi_test_data, batch_size)
#固定写法，可以牢记，cuda代表Gpu
# torch.cuda.is_available()可以查看当前Gpu是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练模型，因为这里是英文数据集，需要用在英文上的预训练模型：bert-base-uncased
# uncased指该预训练模型对应的词表不区分字母的大小写
# 详情可了解：https://huggingface.co/bert-base-uncased
pretrained_model_name = 'xlm-roberta-base'
# 创建模型 BertSST2Model
model = MyXLM.MyXlmModel(categories, pretrained_model_name)
# 固定写法，将模型加载到device上，
# 如果是GPU上运行，此时可以观察到GPU的显存增加
model.to(device)

# 训练过程
# Adam是最近较为常用的优化器，详情可查看：https://www.jianshu.com/p/aebcaf8af76e
optimizer = Adam(model.parameters(), learning_rate)  # 使用Adam优化器
loss = nn.CrossEntropyLoss()  # 使用crossentropy作为二分类任务的损失函数

# 记录当前训练时间，用以记录日志和存储
timestamp = time.strftime("%m_%d_%H_%M", time.localtime())


In [5]:
file_name = 'readme.md'
model_save_path = 'D:/Experiment_models_save/XLM-Roberta-base/hasoc2020/taskA.pth'
en_temp_best = 0.88
de_temp_best = 0.85
hi_temp_best = 0.6
for epoch in range(1, num_epoch + 1):
    MyXLM.train(model, en_train_iter, device, optimizer, loss, epoch)
    MyXLM.train(model, de_train_iter, device, optimizer, loss, epoch)
    MyXLM.train(model, hi_train_iter, device, optimizer, loss, epoch)
    en_acc_score = MyXLM.test(model, en_test_iter, device, epoch, file_name)
    de_acc_score = MyXLM.test(model, de_test_iter, device, epoch, file_name)
    hi_acc_score = MyXLM.test(model, hi_test_iter, device, epoch, file_name)
    print('\n\n')
    if en_acc_score > en_temp_best and de_acc_score > de_temp_best and hi_acc_score > hi_temp_best:
        en_temp_best, de_temp_best, hi_temp_best = en_acc_score, de_acc_score, hi_acc_score
        MyXLM.save_pretrained(model, model_save_path)
        print(f'best en_acc_socre : {en_acc_score}, best de_acc_score : {de_acc_score}, best hi_acc_score : {hi_acc_score}')
print(f'best en_acc_socre : {en_temp_best}, best de_acc_score : {de_temp_best}, best de_acc_score : {hi_temp_best}')


Training Epoch 1:   0%|          | 0/116 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 8.00 GiB total capacity; 5.53 GiB already allocated; 0 bytes free; 6.74 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF